In [177]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#matplotlib.inline


## Sprint ディープラーニングフレームワーク2

### 2.公式Example


深層学習フレームワークには公式に様々なモデルのExampleコードが公開されています。

#### 【問題1】公式チュートリアルモデルを分担して実行
--------------
TensorFLowの公式チュートリアルモデルを分担して実行してください。

以下の中から1人ひとつ選び実行し、その結果を簡単に発表してください。

[models/tutorials at master · tensorflow/models](https://www.tensorflow.org/tutorials/)

[テンソルと演算](https://www.tensorflow.org/tutorials/customization/basics#tensorflow%E3%81%AE%E3%82%A4%E3%83%B3%E3%83%9D%E3%83%BC%E3%83%88)

これは、下記の手法を示す TensorFlow の入門チュートリアルです。

    必要なパッケージのインポート
    テンソルの作成と使用
    GPUによる高速化の使用
    tf.data.Datasetのデモ



In [ ]:
# pip install -q tensorflow-gpu==2.0.0-rc1


上記のコマンドを実行できなかったので下記のリンクで行う


[tensorflow_macos ](https://github.com/apple/tensorflow_macos/issues/153)

macbook Air inter big sur 　conda環境で動作確認済み

### TensorFlowのインポート
はじめに、tensorflow モジュールをインポートします。TensorFlow 2.0 では、eager execution が既定でオンとなっています。 これにより、TensorFlow のフロントエンドがよりインタラクティブになります。詳細は後述します。


In [13]:
import tensorflow-macos  as tf

SyntaxError: invalid syntax (<ipython-input-13-35bc9c9d47d4>, line 1)

In [11]:
tf.__version__

'2.4.0-rc0'

### テンソル
テンソルは多次元配列です。NumPy の ndarray オブジェクトと同様に、tf.Tensor にはデータ型と形状があります。これに加えて、tf.Tensor は（ GPU のような）アクセラレータのメモリに置くことができます。TensorFlow には、tf.Tensor を使用し生成するたくさんの演算(tf.add, tf.matmul, tf.linalg.inv など)のライブラリが存在します。これらの演算では、ネイティブな Python データ型が自動変換されます。例を示します。

In [2]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))


tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


それぞれのtf.Tensorには、形状とデータ型があります。

In [3]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)


tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


NumPy 配列と tf.Tensor の間のもっとも明確な違いは

    テンソルは（ GPU や TPU などの）アクセラレータメモリを使用できる
    テンソルは変更不可

### NumPy互換性

TensorFlow のtf.Tensorと NumPy の ndarray 間の変換は簡単です。

    TensorFlow の演算により NumPy の ndarray は自動的にテンソルに変換される
    NumPy の演算によりテンソルは自動的に NuｍPy の ndarray に変換される

テンソルは .numpy() メソッドを使って明示的に NumPy の ndarray に変換されます。NumPy のndarray と tf.Tensor はその下敷きとなるメモリ上の表現が、できるかぎり共通化されているので、通常この変換のコストは小さいです。しかし、NumPy 配列はホスト側のメモリに置かれる一方、tf.Tensor はGPU のメモリに置かれる可能性もあるため、下層の表現をいつも共通化できるとは限りません。また、変換にはGPU からホスト側メモリへのコピーも関わってきます。

In [4]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow演算によりnumpy配列は自動的にテンソルに変換される")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("またNumPy演算によりテンソルは自動的にnumpy配列に変換される")
print(np.add(tensor, 1))

print(".numpy()メソッドによりテンソルは明示的にnumpy配列に変換される")
print(tensor.numpy())


TensorFlow演算によりnumpy配列は自動的にテンソルに変換される
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
またNumPy演算によりテンソルは自動的にnumpy配列に変換される
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
.numpy()メソッドによりテンソルは明示的にnumpy配列に変換される
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


### GPU による高速化
TensorFlow の演算の多くは、GPU を計算に使用することで高速化されます。TensorFlow は演算に注釈をつけなくとも、自動的に GPU と CPU のどちらかを選択し、必要であればテンソルを GPU メモリと CPU メモリの間でコピーして実行します。演算で生成されたテンソルは通常演算を実行したデバイスのメモリに置かれます。例を見てみましょう。

In [17]:
x = tf.random.uniform([3, 3])

print("利用できるGPUはあるか: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("テンソルはGPU #0にあるか:  "),
print(x.device.endswith('GPU:0'))


利用できるGPUはあるか: 
[]
テンソルはGPU #0にあるか:  
False


上記は多分nvidiaなどのグラフィックボード用のコードだと推測される
今回はmac用のものをインストールできているので次に進む

### デバイス名

Tensor.device プロパティにより、そのテンソルの内容を保持しているデバイスの完全な名前文字列を得ることができます。この名前には、プログラムを実行中のホストのネットワークアドレスや、ホスト上のデバイスについての詳細がエンコードされています。この情報は、TensorFlow プログラムの分散実行に必要なものです。テンソルがホスト上の N 番目のGPUにある場合、文字列の最後は GPU:<N> となります。
### 明示的デバイス配置
TensorFlowでいう配置は、個々の演算を実行するためにどのようにデバイスにアサイン（配置）されるかを指します。前述のとおり、明示的な示唆がなければ、TensorFlow は演算を実行するデバイスを自動的に決め、必要であればテンソルをそのデバイスにコピーします。しかし、tf.device コンテキストマネジャーを使うことで、TensorFlow の演算を特定のデバイスに配置することができます。例を見てみましょう。

In [25]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start

  print("10 loops: {:0.2f}ms".format(1000*result))

# CPUでの実行を強制
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# GPU #0があればその上での実行を強制
if tf.config.experimental.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"): # 2番めのGPUなら GPU:1, 3番目なら GPU:2 など
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)


On CPU:
10 loops: 492.30ms


### データセット

このセクションでは tf.data.Dataset API を使って、モデルにデータを供給するためのパイプラインを構築します。tf.data.Dataset APIは、単純で再利用可能な部品をもとに、モデルの訓練あるいは評価ループにデータを供給する高性能で複雑な入力パイプラインを構築するために使われます。
### ソースDatasetの作成
Dataset.from_tensors やDataset.from_tensor_slices といったファクトリー関数または TextLineDataset あるいはTFRecordDataset のようなファイルを読み込むオブジェクトを使って、 元となるデータセットを作成しましょう。詳しくは、TensorFlow Dataset guide を参照してください。

In [7]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# CSVファイルを作成
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)


### 変換の適用
map, batch, shuffle などの変換関数を使って、データセットレコードに変換を適用します。

In [9]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)


In [10]:
print('ds_tensors の要素:')
for x in ds_tensors:
  print(x)

print('\nds_file の要素:')
for x in ds_file:
  print(x)


ds_tensors の要素:
tf.Tensor([1 9], shape=(2,), dtype=int32)
tf.Tensor([ 4 16], shape=(2,), dtype=int32)
tf.Tensor([36 25], shape=(2,), dtype=int32)

ds_file の要素:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)


#### 【問題2】（アドバンス課題）様々な手法を実行
----------------
TensorFLowやGoogle AI ResearchのGitHubリポジトリには、定番のモデルから最新のモデルまで多様なコードが公開されています。これらから興味あるものを選び実行してください。

なお、これらのコードは初学者向けではないため、巨大なデータセットのダウンロードが必要な場合など、実行が簡単ではないこともあります。そういった場合は、コードリーディングを行ってください。

[models/research at master · tensorflow/models](https://github.com/tensorflow/models/tree/master/research)

[google-research/google-research: Google AI Research](https://github.com/google-research/google-research)

更新日が古いものはPythonやTensorFlowのバージョンが古く、扱いずらい場合があります。新しいものから見ることを推奨します。

### 3.異なるフレームワークへの書き換え


「ディープラーニングフレームワーク1」で作成した4種類のデータセットを扱うTensorFLowのコードを異なるフレームワークに変更していきます。

Iris（Iris-versicolorとIris-virginicaのみの2値分類）
Iris（3種類全ての目的変数を使用して多値分類）
House Prices
MNIST

#### Kerasへの書き換え
--------------
KerasはTensorFLowに含まれるtf.kerasモジュールを使用してください。

KerasにはSequentialモデルかFunctional APIかなど書き方に種類がありますが、これは指定しません。

#### 【問題3】Iris（2値分類）をKerasで学習
--------------
TensorFlowによるIrisデータセットに対する2値分類をKerasに書き換えてください。

In [271]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data[:100,:]
y = iris.target[:100]

In [272]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=19)

In [273]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=19)

In [274]:
X_train.shape

(80, 4)

In [275]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, input_dim=4))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(32))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid')) 
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])


In [276]:
model.summary()

Model: "sequential_49"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_81 (Dense)             (None, 64)                320       
_________________________________________________________________
activation_66 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_82 (Dense)             (None, 32)                2080      
_________________________________________________________________
activation_67 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_83 (Dense)             (None, 1)                 33        
_________________________________________________________________
activation_68 (Activation)   (None, 1)                 0         
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
___________________________________________________

In [277]:
history = model.fit(X_train, y_train,
                    batch_size=1,
                    epochs=20,
                    verbose=1,
                    validation_data=(X_val,y_val))

Epoch 1/20
80/80 [==============================] - 1s 4ms/step - loss: 0.4694 - accuracy: 0.7961 - val_loss: 0.1174 - val_accuracy: 1.0000
Epoch 2/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0917 - accuracy: 1.0000 - val_loss: 0.0308 - val_accuracy: 1.0000
Epoch 3/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0317 - accuracy: 1.0000 - val_loss: 0.0188 - val_accuracy: 1.0000
Epoch 4/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0192 - accuracy: 1.0000 - val_loss: 0.0068 - val_accuracy: 1.0000
Epoch 5/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0095 - accuracy: 1.0000 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 6/20
80/80 [==============================] - 0s 3ms/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 7/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 8/20
80/80 [==

In [278]:
model.predict(X_test, batch_size=10, verbose=1)

2/2 [==============================] - 0s 3ms/step


array([[9.9998498e-01],
       [9.9999821e-01],
       [3.7465645e-05],
       [5.9486447e-05],
       [1.7929077e-04],
       [3.6299229e-04],
       [9.9998677e-01],
       [5.5846572e-04],
       [9.9999869e-01],
       [9.9997675e-01],
       [6.3446164e-04],
       [4.7916174e-04],
       [9.9999189e-01],
       [2.4646521e-04],
       [8.1200618e-05],
       [8.3259285e-05],
       [8.7203240e-05],
       [9.9999136e-01],
       [5.8486723e-05],
       [9.9999648e-01]], dtype=float32)

In [279]:
model.metrics_names
model.evaluate(X_test,y_test, batch_size=10)

2/2 [==============================] - 0s 5ms/step - loss: 1.4717e-04 - accuracy: 1.0000


[0.0001471704599680379, 1.0]

#### 【問題4】Iris（多値分類）をKerasで学習
----------------------
TensorFlowによるIrisデータセットに対する3値分類をKerasに書き換えてください。

In [91]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

In [92]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=19)
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=19)

In [93]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(12, input_dim=4))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(3, input_dim=12))
model.add(tf.keras.layers.Activation('softmax')) 
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [94]:
history = model.fit(X_train, y_train,
                    batch_size=1,
                    epochs=20,
                    verbose=1,
                    validation_data=(X_val,y_val))

Epoch 1/20
120/120 [==============================] - 2s 7ms/step - loss: 1.4113 - accuracy: 0.3732 - val_loss: 1.0598 - val_accuracy: 0.4000
Epoch 2/20
120/120 [==============================] - 0s 4ms/step - loss: 1.1030 - accuracy: 0.4197 - val_loss: 0.8874 - val_accuracy: 0.5667
Epoch 3/20
120/120 [==============================] - 0s 3ms/step - loss: 0.8960 - accuracy: 0.5102 - val_loss: 0.8023 - val_accuracy: 0.4667
Epoch 4/20
120/120 [==============================] - 1s 5ms/step - loss: 0.7620 - accuracy: 0.5272 - val_loss: 0.7207 - val_accuracy: 0.4333
Epoch 5/20
120/120 [==============================] - 0s 3ms/step - loss: 0.6886 - accuracy: 0.6637 - val_loss: 0.6509 - val_accuracy: 0.7667
Epoch 6/20
120/120 [==============================] - 0s 3ms/step - loss: 0.6302 - accuracy: 0.8502 - val_loss: 0.5835 - val_accuracy: 0.8333
Epoch 7/20
120/120 [==============================] - 0s 2ms/step - loss: 0.5598 - accuracy: 0.8587 - val_loss: 0.5384 - val_accuracy: 0.8333
Epoch 

In [95]:
model.predict(X_test, batch_size=10, verbose=1)

3/3 [==============================] - 0s 1ms/step


array([[9.34664130e-01, 6.37958050e-02, 1.54001685e-03],
       [3.66249995e-04, 2.77208716e-01, 7.22425044e-01],
       [7.95551017e-03, 5.21336496e-01, 4.70707923e-01],
       [2.18699947e-02, 6.17851138e-01, 3.60278964e-01],
       [9.46399927e-01, 5.27066700e-02, 8.93365243e-04],
       [9.42205369e-01, 5.66952378e-02, 1.09934469e-03],
       [9.27082956e-01, 7.08366260e-02, 2.08051968e-03],
       [9.42058265e-01, 5.68346679e-02, 1.10706466e-03],
       [1.27738984e-02, 5.67667365e-01, 4.19558704e-01],
       [4.29460088e-05, 1.86832651e-01, 8.13124418e-01],
       [1.52144935e-02, 5.34833848e-01, 4.49951649e-01],
       [9.19470310e-01, 7.85493925e-02, 1.98032125e-03],
       [7.25826249e-03, 5.53274632e-01, 4.39467072e-01],
       [9.37170327e-01, 6.14704341e-02, 1.35920849e-03],
       [5.39399334e-04, 3.00176352e-01, 6.99284315e-01],
       [9.26956713e-01, 7.09528700e-02, 2.09037890e-03],
       [8.58111889e-05, 1.84155703e-01, 8.15758526e-01],
       [9.56743121e-01, 4.27623

In [96]:
model.metrics_names
model.evaluate(X_test,y_test, batch_size=10)

3/3 [==============================] - 0s 4ms/step - loss: 0.2899 - accuracy: 1.0000


[0.28991222381591797, 1.0]

#### 【問題5】House PricesをKerasで学習
------------------
TensorFlowによるHouse Pricesデータセットに対する回帰をKerasに書き換えてください。


In [316]:
df = pd.read_csv("../../term1/Sprint/train.csv")
X = df[["YearBuilt","GrLivArea"]]
y = df[["SalePrice"]]

In [317]:
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)

In [318]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X = mms.fit_transform(X)
y = mms.fit_transform(y)

In [319]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [320]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, input_dim=X.shape[1]))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(32))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('relu')) 
model.compile(optimizer='RMSProp',loss='mean_squared_error',metrics=['mae'])


In [321]:
history = model.fit(X_train, y_train,
                    batch_size=1,
                    epochs=5,
                    verbose=1,
                    validation_data=(X_val,y_val))

Epoch 1/5
934/934 [==============================] - 2s 2ms/step - loss: 0.0181 - mae: 0.0894 - val_loss: 0.0032 - val_mae: 0.0377
Epoch 2/5
934/934 [==============================] - 2s 2ms/step - loss: 0.0046 - mae: 0.0471 - val_loss: 0.0041 - val_mae: 0.0498
Epoch 3/5
934/934 [==============================] - 2s 2ms/step - loss: 0.0045 - mae: 0.0455 - val_loss: 0.0037 - val_mae: 0.0474
Epoch 4/5
934/934 [==============================] - 2s 2ms/step - loss: 0.0047 - mae: 0.0467 - val_loss: 0.0030 - val_mae: 0.0378
Epoch 5/5
934/934 [==============================] - 2s 3ms/step - loss: 0.0038 - mae: 0.0431 - val_loss: 0.0028 - val_mae: 0.0375


In [322]:
model.predict(X_test, batch_size=10, verbose=1)

30/30 [==============================] - 0s 1ms/step


array([[0.30630293],
       [0.15522876],
       [0.12248225],
       [0.2654441 ],
       [0.13500974],
       [0.06374262],
       [0.23222509],
       [0.12878211],
       [0.6185987 ],
       [0.18402173],
       [0.21718492],
       [0.21399179],
       [0.295238  ],
       [0.10838627],
       [0.10703231],
       [0.14197531],
       [0.27489737],
       [0.15584551],
       [0.14350724],
       [0.16271447],
       [0.14468254],
       [0.15492292],
       [0.0939771 ],
       [0.21503676],
       [0.23892067],
       [0.11150587],
       [0.23692353],
       [0.0850068 ],
       [0.2659527 ],
       [0.12083102],
       [0.19897167],
       [0.25497302],
       [0.12427593],
       [0.32956475],
       [0.31921667],
       [0.21379876],
       [0.2430643 ],
       [0.11294258],
       [0.31853807],
       [0.38587442],
       [0.24512976],
       [0.14500585],
       [0.21064016],
       [0.28110838],
       [0.41626233],
       [0.20154244],
       [0.08583949],
       [0.119

In [335]:
model.metrics_names
model.evaluate(X_test,y_test, batch_size=10)

30/30 [==============================] - 0s 2ms/step - loss: 0.0058 - mae: 0.0498


[0.0058202496729791164, 0.049791280180215836]

#### 【問題6】MNISTをKerasで学習
------------------
TensorFlowによるMNISTデータセットによる画像の多値分類をKerasに書き換えてください。

In [166]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


In [167]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(128,activation='relu'))

model.add(tf.keras.layers.Dense(10, activation='softmax'))


In [168]:
predictions = model(x_train[:1]).numpy()
predictions

array([[0.09310647, 0.06202954, 0.08150596, 0.12916912, 0.13066141,
        0.0819173 , 0.03002794, 0.09720318, 0.16176642, 0.13261269]],
      dtype=float32)

In [169]:
tf.nn.softmax(predictions).numpy()


array([[0.09924501, 0.09620821, 0.09810037, 0.10288937, 0.10304302,
        0.09814073, 0.09317814, 0.09965242, 0.10629854, 0.10324428]],
      dtype=float32)

In [170]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [171]:
loss_fn(y_train[:1], predictions).numpy()


2.321353

In [172]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])


In [173]:
model.fit(x_train, y_train, epochs=5)


Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4277 - accuracy: 0.8790
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1157 - accuracy: 0.9661
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0780 - accuracy: 0.9764
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0557 - accuracy: 0.9834
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0429 - accuracy: 0.9864


In [174]:
model.evaluate(x_test,  y_test, verbose=2)


313/313 - 1s - loss: 0.0789 - accuracy: 0.9740


[0.07888893783092499, 0.9739999771118164]

#### 【問題7】（アドバンス課題）PyTorchへの書き換え
-----------------
4種類の問題をPyTorchに書き換えてください。

In [34]:
import torch

In [33]:
iris = load_iris()
X = iris.data[:100,:]
y = iris.target[:100]

In [35]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=19)

In [37]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(y_val).long()

In [40]:
X_train.shape

torch.Size([80, 4])

In [42]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

# indexを指定すればデータを取り出す
index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

torch.Size([4])
tensor(1)


In [44]:
from torch.utils.data import DataLoader

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# イテレータに変換
batch_iterator = iter(train_dataloader)
# 1番目の要素を取り出す
inputs, labels = next(batch_iterator)
print(inputs.size())
print(labels.size())

torch.Size([32, 4])
torch.Size([32])


In [46]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=4, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=2, bias=True)
)


In [47]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

In [48]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

In [50]:
# エポック数
num_epochs = 50

# 学習時と検証時で分けるためディクショナリを用意
dataloaders_dict = {
    'train': train_dataloader,
    'val': val_dataloader
}

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-------------')
    
    for phase in ['train', 'val']:
        
        if phase == 'train':
            # モデルを訓練モードに設定
            net.train()
        else:
            # モデルを推論モードに設定
            net.eval()
        
        # 損失和
        epoch_loss = 0.0
        # 正解数
        epoch_corrects = 0
        
        # DataLoaderからデータをバッチごとに取り出す
        for inputs, labels in dataloaders_dict[phase]:
            
            # optimizerの初期化
            optimizer.zero_grad()
            
            # 学習時のみ勾配を計算させる設定にする
            with torch.set_grad_enabled(phase == 'train'):
                outputs = net(inputs)
                
                # 損失を計算
                loss = criterion(outputs, labels)
                
                # ラベルを予測
                _, preds = torch.max(outputs, 1)
                
                # 訓練時はバックプロパゲーション
                if phase == 'train':
                    # 逆伝搬の計算
                    loss.backward()
                    # パラメータの更新
                    optimizer.step()
                
                # イテレーション結果の計算
                # lossの合計を更新
                # PyTorchの仕様上各バッチ内での平均のlossが計算される。
                # データ数を掛けることで平均から合計に変換をしている。
                # 損失和は「全データの損失/データ数」で計算されるため、
                # 平均のままだと損失和を求めることができないため。
                epoch_loss += loss.item() * inputs.size(0)
                
                # 正解数の合計を更新
                epoch_corrects += torch.sum(preds == labels.data)

        # epochごとのlossと正解率を表示
        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))


Epoch 1/50
-------------
train Loss: 0.7158 Acc: 0.5250
val Loss: 0.7290 Acc: 0.4000
Epoch 2/50
-------------
train Loss: 0.6999 Acc: 0.5250
val Loss: 0.7227 Acc: 0.4000
Epoch 3/50
-------------
train Loss: 0.6904 Acc: 0.5250
val Loss: 0.7077 Acc: 0.4000
Epoch 4/50
-------------
train Loss: 0.6813 Acc: 0.5250
val Loss: 0.7041 Acc: 0.4000
Epoch 5/50
-------------
train Loss: 0.6716 Acc: 0.5250
val Loss: 0.7032 Acc: 0.4000
Epoch 6/50
-------------
train Loss: 0.6621 Acc: 0.5250
val Loss: 0.6860 Acc: 0.4000
Epoch 7/50
-------------
train Loss: 0.6529 Acc: 0.5250
val Loss: 0.6729 Acc: 0.4000
Epoch 8/50
-------------
train Loss: 0.6433 Acc: 0.5250
val Loss: 0.6573 Acc: 0.4000
Epoch 9/50
-------------
train Loss: 0.6350 Acc: 0.5375
val Loss: 0.6523 Acc: 0.4000
Epoch 10/50
-------------
train Loss: 0.6270 Acc: 0.5375
val Loss: 0.6409 Acc: 0.4000
Epoch 11/50
-------------
train Loss: 0.6258 Acc: 0.5375
val Loss: 0.6201 Acc: 1.0000
Epoch 12/50
-------------
train Loss: 0.6100 Acc: 0.9875
val Lo

#### 【問題8】（アドバンス課題）フレームワークの比較
------------------
それぞれのフレームワークにはどのような違いがあるかをまとめてください。

《視点例》

計算速度
コードの行数・可読性
用意されている機能

#### ２日目の発表について
---------------------------
今回のsprintでは１日目の夕方の発表を通常通り１名の方に行っていただき、２日目にはtensorflowのチュートリアルについての発表を全員に行っていただきます。 
目安としての発表時間は5分-10分程度です。

発表形式の例：
チュートリアルの概要（どういうものか、ゴールは何か）
これを選んだ理由
コードリーディング・デバッグで学んだこと（一番難しかったところ、応用できそうなところ）

・keras シンプルで使いやすい  
・pytorch 使いづらいが拡張性が高い  
・tensorflow  使いづら過ぎるが、使用率が高い  